# Exercises
insert some Batch Normalization layers into the generator and discriminator networks and see if it helps training.

## Create training data

In [1]:
from keras.datasets import mnist
import numpy as np

# load the mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# normalize the data
x_train = x_train.astype('float32') / 255 *2 -1
x_test = x_test.astype('float32') / 255 *2 -1

# reshape the data
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

2023-03-06 16:58:37.626899: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Setting up the hyperparameters

In [2]:
n_learn = 10001 # number of learning
interval = 1000 # interval of learning
batch_size = 32 # batch size
n_noize = 128 # number of noize
img_size = 28 # image size
alpha = 0.2 # alpha for leaky relu

# set up the optimizer
from keras.optimizers import Adam
opt = Adam(lr=0.0002, beta_1=0.5)

/Users/takamatsunaoki/opt/anaconda3/envs/ai-master/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


## Generator
Activation in hidden layer:  LeakyReLU

In [3]:
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, BatchNormalization

# generator
generator = Sequential()
generator.add(Dense(256, input_dim=n_noize))
generator.add(LeakyReLU(alpha=alpha))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=alpha))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=alpha))
generator.add(Dense(img_size*img_size, activation='tanh'))
generator.summary()

2023-03-06 17:02:28.794670: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               33024     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 1024)              525312    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 784)               8

## Discriminator
Activation in hidden layer:  LeakyReLU
loss:  binary_crossentropy

In [4]:
# discriminator
discriminator = Sequential()
discriminator.add(Dense(512, input_dim=img_size*img_size))
discriminator.add(LeakyReLU(alpha=alpha))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(alpha=alpha))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.summary()

# compile the discriminator
discriminator.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


## Combine the generator and discriminator

In [7]:
from keras.layers import Input
from keras.models import Model

# training the discriminator only when training the GAN
discriminator.trainable = False

# the discriminator determines the image created by the generator for the noise
noize = Input(shape=(n_noize,))
img = generator(noize)
reality = discriminator(img)

# combine the generator and the discriminator
comb = Model(noize, reality)
comb.compile(loss='binary_crossentropy', optimizer=opt)
comb.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128)]             0         
                                                                 
 sequential (Sequential)     (None, 784)               1493520   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 533505    
                                                                 
Total params: 2,027,025
Trainable params: 1,493,520
Non-trainable params: 533,505
_________________________________________________________________


## Create the images

In [8]:
import matplotlib.pyplot as plt
def gerenate_img(i):
    # generate the image
    n_row = 5
    n_col = 5
    noize = np.random.normal(0, 1, (n_row*n_col, n_noize))
    gen_imgs = generator.predict(noize)
    gen_imgs = 0.5 * gen_imgs + 0.5

    matrix_image = np.zeros((img_size*n_row, img_size*n_col))

    # arrange the generated images into a single image
    for j in range(n_row):
        for k in range(n_col):
            matrix_image[j*img_size:(j+1)*img_size, k*img_size:(k+1)*img_size] = gen_imgs[j*n_col+k].reshape(img_size, img_size)
    
    # show the image
    plt.figure(figsize=(10, 10))
    plt.imshow(matrix_image, cmap='Greys_r')
    plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
    plt.show()

## Training

In [ ]:
# set up the learning
batch_half = int(batch_size / 2)
loss_record = np.zeros((n_learn, 3))
acc_record = np.zeros((n_learn, 2))

# learning
for i in range(n_learn):
    # train the discriminator with the generated images
    g_noize = np.random.normal(0, 1, (batch_half, n_noize))
    g_imgs = generator.predict(g_noize)
    loss_fake, acc_fake = discriminator.train_on_batch(g_imgs, np.zeros((batch_half, 1)))
    loss_record[i, 0] = loss_fake
    acc_record[i, 0] = acc_fake

    # train the discriminator with the real images
    idx = np.random.randint(0, x_train.shape[0], batch_half)
    real_imgs = x_train[idx, :]
    loss_real, acc_real = discriminator.train_on_batch(real_imgs, np.ones((batch_half, 1)))
    loss_record[i, 1] = loss_real
    acc_record[i, 1] = acc_real

    # train the generator withe the combined model
    c_noize = np.random.normal(0, 1, (batch_size, n_noize))
    loss_comb = comb.train_on_batch(c_noize, np.ones((batch_size, 1)))
    loss_record[i, 2] = loss_comb

    # show the learning process
    if i % interval == 0:
        print('epoch: %d, loss_fake: %f, loss_real: %f, loss_comb: %f, acc_fake: %f, acc_real: %f' % (i, loss_fake, loss_real, loss_comb, acc_fake, acc_real))
        gerenate_img(i)

## History of the loss and the accuracy

In [ ]:
# history of the loss
n_plt_loss = 500
plt.plot(np.arange(n_plt_loss), loss_record[:n_plt_loss, 0], label='loss_fake')
plt.plot(np.arange(n_plt_loss), loss_record[:n_plt_loss, 1], label='loss_real')
plt.plot(np.arange(n_plt_loss), loss_record[:n_plt_loss, 2], label='loss_comb')
plt.legend()
plt.title('Loss')
plt.show()

# history of the accuracy
n_plt_acc = 1000
plt.plot(np.arange(n_plt_acc), acc_record[:n_plt_acc, 0], label='acc_fake')
plt.plot(np.arange(n_plt_acc), acc_record[:n_plt_acc, 1], label='acc_real')
plt.legend()
plt.title('Accuracy')
plt.show()